In [2]:
{
 "cells": [
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "# 02 - Tuning Parametri HSV\n",
    "\n",
    "Notebook interattivo per ottimizzare i parametri di detection HSV.\n",
    "\n",
    "**Obiettivi:**\n",
    "- Visualizzare lo spazio HSV del frame\n",
    "- Testare interattivamente range HSV\n",
    "- Trovare i migliori valori per il tuo scenario"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "import sys\n",
    "sys.path.insert(0, '..')\n",
    "\n",
    "import cv2\n",
    "import numpy as np\n",
    "import matplotlib.pyplot as plt\n",
    "from ipywidgets import interact, IntSlider, FloatSlider\n",
    "from pathlib import Path\n",
    "\n",
    "%matplotlib inline"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 1. Carica Frame"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "VIDEO_PATH = '../data/videos/input/video1.mp4'  # MODIFICA\n",
    "\n",
    "cap = cv2.VideoCapture(VIDEO_PATH)\n",
    "ret, frame = cap.read()\n",
    "cap.release()\n",
    "\n",
    "if ret:\n",
    "    print(f\"✓ Frame caricato: {frame.shape}\")\n",
    "    hsv_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)\n",
    "else:\n",
    "    print(\"❌ Errore caricamento\")"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 2. Visualizza Canali HSV"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "fig, axes = plt.subplots(2, 2, figsize=(14, 10))\n",
    "\n",
    "# Frame originale\n",
    "axes[0, 0].imshow(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))\n",
    "axes[0, 0].set_title('Originale (RGB)')\n",
    "axes[0, 0].axis('off')\n",
    "\n",
    "# Canale H (Hue)\n",
    "axes[0, 1].imshow(hsv_frame[:,:,0], cmap='hsv')\n",
    "axes[0, 1].set_title('Hue (H)')\n",
    "axes[0, 1].axis('off')\n",
    "\n",
    "# Canale S (Saturation)\n",
    "axes[1, 0].imshow(hsv_frame[:,:,1], cmap='gray')\n",
    "axes[1, 0].set_title('Saturation (S)')\n",
    "axes[1, 0].axis('off')\n",
    "\n",
    "# Canale V (Value)\n",
    "axes[1, 1].imshow(hsv_frame[:,:,2], cmap='gray')\n",
    "axes[1, 1].set_title('Value (V)')\n",
    "axes[1, 1].axis('off')\n",
    "\n",
    "plt.tight_layout()\n",
    "plt.show()\n",
    "\n",
    "# Statistiche HSV\n",
    "print(\"\\nStatistiche HSV:\")\n",
    "print(f\"  H range: [{hsv_frame[:,:,0].min()}, {hsv_frame[:,:,0].max()}]\")\n",
    "print(f\"  S range: [{hsv_frame[:,:,1].min()}, {hsv_frame[:,:,1].max()}]\")\n",
    "print(f\"  V range: [{hsv_frame[:,:,2].min()}, {hsv_frame[:,:,2].max()}]\")"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 3. Tuning Interattivo Range Rosso"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "def test_red_range(h_lower1, h_upper1, h_lower2, h_upper2, s_lower, v_lower):\n",
    "    \"\"\"\n",
    "    Test interattivo range rosso.\n",
    "    Il rosso in HSV è split: 0-10 e 170-180\n",
    "    \"\"\"\n",
    "    # Range 1 (0-10)\n",
    "    lower1 = np.array([h_lower1, s_lower, v_lower])\n",
    "    upper1 = np.array([h_upper1, 255, 255])\n",
    "    mask1 = cv2.inRange(hsv_frame, lower1, upper1)\n",
    "    \n",
    "    # Range 2 (170-180)\n",
    "    lower2 = np.array([h_lower2, s_lower, v_lower])\n",
    "    upper2 = np.array([h_upper2, 255, 255])\n",
    "    mask2 = cv2.inRange(hsv_frame, lower2, upper2)\n",
    "    \n",
    "    # Combina\n",
    "    mask = cv2.bitwise_or(mask1, mask2)\n",
    "    \n",
    "    # Morfologia\n",
    "    kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (5,5))\n",
    "    mask = cv2.morphologyEx(mask, cv2.MORPH_OPEN, kernel)\n",
    "    mask = cv2.morphologyEx(mask, cv2.MORPH_CLOSE, kernel)\n",
    "    \n",
    "    # Visualizza\n",
    "    fig, axes = plt.subplots(1, 3, figsize=(18, 6))\n",
    "    \n",
    "    axes[0].imshow(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))\n",
    "    axes[0].set_title('Originale')\n",
    "    axes[0].axis('off')\n",
    "    \n",
    "    axes[1].imshow(mask, cmap='gray')\n",
    "    axes[1].set_title(f'Maschera (Pixel: {np.sum(mask > 0)})')\n",
    "    axes[1].axis('off')\n",
    "    \n",
    "    # Overlay\n",
    "    overlay = frame.copy()\n",
    "    overlay[mask > 0] = [0, 255, 0]\n",
    "    result = cv2.addWeighted(frame, 0.7, overlay, 0.3, 0)\n",
    "    axes[2].imshow(cv2.cvtColor(result, cv2.COLOR_BGR2RGB))\n",
    "    axes[2].set_title('Overlay')\n",
    "    axes[2].axis('off')\n",
    "    \n",
    "    plt.tight_layout()\n",
    "    plt.show()\n",
    "    \n",
    "    print(f\"Range 1: H=[{h_lower1}, {h_upper1}], S>={s_lower}, V>={v_lower}\")\n",
    "    print(f\"Range 2: H=[{h_lower2}, {h_upper2}], S>={s_lower}, V>={v_lower}\")\n",
    "\n",
    "# Widget interattivi\n",
    "interact(\n",
    "    test_red_range,\n",
    "    h_lower1=IntSlider(min=0, max=10, step=1, value=0, description='H Lower 1'),\n",
    "    h_upper1=IntSlider(min=0, max=20, step=1, value=10, description='H Upper 1'),\n",
    "    h_lower2=IntSlider(min=160, max=180, step=1, value=170, description='H Lower 2'),\n",
    "    h_upper2=IntSlider(min=170, max=180, step=1, value=180, description='H Upper 2'),\n",
    "    s_lower=IntSlider(min=0, max=255, step=5, value=100, description='S Lower'),\n",
    "    v_lower=IntSlider(min=0, max=255, step=5, value=100, description='V Lower')\n",
    ");"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 4. Tuning Interattivo Range Bianco"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "def test_white_range(s_upper, v_lower):\n",
    "    \"\"\"\n",
    "    Test interattivo range bianco.\n",
    "    Il bianco ha Saturation bassa e Value alta.\n",
    "    \"\"\"\n",
    "    lower = np.array([0, 0, v_lower])\n",
    "    upper = np.array([180, s_upper, 255])\n",
    "    mask = cv2.inRange(hsv_frame, lower, upper)\n",
    "    \n",
    "    # Morfologia\n",
    "    kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (5,5))\n",
    "    mask = cv2.morphologyEx(mask, cv2.MORPH_OPEN, kernel)\n",
    "    mask = cv2.morphologyEx(mask, cv2.MORPH_CLOSE, kernel)\n",
    "    \n",
    "    # Visualizza\n",
    "    fig, axes = plt.subplots(1, 3, figsize=(18, 6))\n",
    "    \n",
    "    axes[0].imshow(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))\n",
    "    axes[0].set_title('Originale')\n",
    "    axes[0].axis('off')\n",
    "    \n",
    "    axes[1].imshow(mask, cmap='gray')\n",
    "    axes[1].set_title(f'Maschera (Pixel: {np.sum(mask > 0)})')\n",
    "    axes[1].axis('off')\n",
    "    \n",
    "    # Overlay\n",
    "    overlay = frame.copy()\n",
    "    overlay[mask > 0] = [0, 255, 0]\n",
    "    result = cv2.addWeighted(frame, 0.7, overlay, 0.3, 0)\n",
    "    axes[2].imshow(cv2.cvtColor(result, cv2.COLOR_BGR2RGB))\n",
    "    axes[2].set_title('Overlay')\n",
    "    axes[2].axis('off')\n",
    "    \n",
    "    plt.tight_layout()\n",
    "    plt.show()\n",
    "    \n",
    "    print(f\"Range: H=[0, 180], S<={s_upper}, V>={v_lower}\")\n",
    "\n",
    "# Widget interattivi\n",
    "interact(\n",
    "    test_white_range,\n",
    "    s_upper=IntSlider(min=0, max=100, step=5, value=30, description='S Upper'),\n",
    "    v_lower=IntSlider(min=100, max=255, step=5, value=200, description='V Lower')\n",
    ");"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 5. Esporta Parametri Ottimizzati"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Dopo aver trovato i valori ottimali, inseriscili qui\n",
    "OPTIMIZED_PARAMS = {\n",
    "    'red': {\n",
    "        'lower1': [0, 100, 100],     # MODIFICA CON I TUOI VALORI\n",
    "        'upper1': [10, 255, 255],\n",
    "        'lower2': [170, 100, 100],\n",
    "        'upper2': [180, 255, 255]\n",
    "    },\n",
    "    'white': {\n",
    "        'lower': [0, 0, 200],\n",
    "        'upper': [180, 30, 255]\n",
    "    }\n",
    "}\n",
    "\n",
    "print(\"Parametri ottimizzati:\")\n",
    "print(OPTIMIZED_PARAMS)\n",
    "print(\"\\nAggiorna questi valori in config/detection_params.yaml\")"
   ]
  }
 ],
 "metadata": {
  "kernelspec": {
   "display_name": "Python 3",
   "language": "python",
   "name": "python3"
  },
  "language_info": {
   "codemirror_mode": {
    "name": "ipython",
    "version": 3
   },
   "file_extension": ".py",
   "mimetype": "text/x-python",
   "name": "python",
   "nbconvert_exporter": "python",
   "pygments_lexer": "ipython3",
   "version": "3.10.0"
  }
 },
 "nbformat": 4,
 "nbformat_minor": 4
}

NameError: name 'null' is not defined